In [301]:
#This notebook details the initial cleaning and proccessing of raw data for sure in analysis. See main.ipynb for the empirical work

import pandas as pd
#Importing Abbotsford data set
df=pd.read_csv("data/Abbotsford.csv")
df.head()


,Location,Crash Type,Municipality,Year,Crash Count,Latitude,Longitude
0,STATION RD & SWENSSON AVE,PDO,ABBOTSFORD,2020,1,49.051184,-122.456826
1,LEFEUVRE RD & MARSH MCCORMICK RD,PDO,ABBOTSFORD,2020,2,49.146958,-122.450297
2,FRASER HWY & LEFEUVRE RD & TURNING LANE,PDO,ABBOTSFORD,2020,8,49.057313,-122.448685
3,HUNTINGDON RD & LEFEUVRE RD,PDO,ABBOTSFORD,2020,1,49.016892,-122.448677
4,KING RD & LEFEUVRE RD,PDO,ABBOTSFORD,2020,4,49.031414,-122.448667


In [302]:
#This code block details the initial cleaning of the "abbotsford" data. The process is repeated in a loop in the code block below for all other data sets


#Filter data to remove entries without & (not intersections) and make categorical dummy variables for years.
mask = df['Location'].str.contains('&')
df = df[mask]

#Drop latitude and longitude
df.drop(['Latitude', 'Longitude'], axis=1, inplace=True)


#Filter to only contain crash type "PDO"
mask = df['Crash Type'].str.contains('PDO')
df_pdo = df[mask]

condition = 20 #CDrop intersections with fewer than 20 PDO crashes in 5 year period (very lighlly trafficed, can't be compared to treatment


#Find all intersection names
unique_names = df_pdo['Location'].unique()
output = [None]*len(unique_names)
for i in range(len(unique_names)):
    output[i] = df_pdo.loc[df['Location'] == unique_names[i], 'Crash Count'].sum()

#temporary data frame that stores names to be filtered
temp = pd.DataFrame({'Unique Names': unique_names, 'Total Crashes': output})
temp['Condition'] = temp['Total Crashes'] >= condition

#filtering rows containing these names from the df_pdo
names = temp.loc[~temp['Condition'], 'Unique Names']
df_pdo = df_pdo[~df_pdo['Location'].isin(names)]

#Filter to only contain crash type "Casualty" and only include intersections that were included under the PDO criteria
mask = df['Crash Type'].str.contains('Casualty')
df_casualty = df[mask]
df_casualty = df_casualty[df_casualty['Location'].isin(df_pdo['Location'])]

#Finds missing rows and imputes 0s for both df_casualty and df_pdo
#Code not at all elegant, just doing it twice

#For PDO
# Create a new DataFrame with all unique combinations of 'Location' and 'Year'
all_combinations_pdo = pd.DataFrame([(location, year) for location in df_pdo['Location'].unique() for year in df_pdo['Year'].unique()], columns=['Location', 'Year'])

# Merge the new DataFrame with the original DataFrame to identify missing combinations
merged_df_pdo = pd.merge(all_combinations_pdo, df_pdo, on=['Location', 'Year'], how='left', indicator=True)

# Filter rows where the original DataFrame had missing data (indicator column is 'left_only')
missing_data_pdo = merged_df_pdo[merged_df_pdo['_merge'] == 'left_only']

# Drop the indicator column
missing_data_pdo = missing_data_pdo.drop(columns=['_merge'])


#Repeat above for casualty
all_combinations_casualty = pd.DataFrame([(location, year) for location in df_casualty['Location'].unique() for year in df_casualty['Year'].unique()], columns=['Location', 'Year'])

# Merge the new DataFrame with the original DataFrame to identify missing combinations
merged_df_casualty = pd.merge(all_combinations_casualty, df_casualty, on=['Location', 'Year'], how='left', indicator=True)

# Filter rows where the original DataFrame had missing data (indicator column is 'left_only')
missing_data_casualty = merged_df_casualty[merged_df_casualty['_merge'] == 'left_only']

# Drop the indicator column
missing_data_casualty = missing_data_casualty.drop(columns=['_merge'])


#Imputing new rows with zero crashes in the relevant 
missing_data_pdo['Crash Type'].fillna('PDO', inplace=True)
missing_data_pdo['Municipality'].fillna('ABBOTSFORD', inplace=True)
missing_data_pdo['Crash Count'].fillna(0, inplace=True)

missing_data_casualty['Crash Type'].fillna('Casualty', inplace=True)
missing_data_casualty['Municipality'].fillna('ABBOTSFORD', inplace=True)
missing_data_casualty['Crash Count'].fillna(0, inplace=True)


#Creating final data frame for Abbotsford with all years for intersections with any property damage only crash
df = pd.concat([df_pdo,df_casualty,missing_data_pdo,missing_data_casualty],axis=0).sort_values(by='Location')

#Pivot df to sort crash counts
pivot_df = df.pivot(index=['Location', 'Year'], columns='Crash Type', values='Crash Count').reset_index()
pivot_df.columns = ['Location', 'Year', 'Casualty', 'PDO']

#Add columns back for municipality, and total number of crashes
pivot_df['Total']=pivot_df['PDO']+pivot_df['Casualty']
pivot_df['Municipality'] = 'ABBOTSFORD'

#Create new data frame that will store all the data - with df just being temporary for construction
df_main = pivot_df

df_main




,Location,Year,Casualty,PDO,Total,Municipality
0,4TH AVE & SUMAS WAY,2018,1.0,2.0,3.0,ABBOTSFORD
1,4TH AVE & SUMAS WAY,2019,2.0,8.0,10.0,ABBOTSFORD
2,4TH AVE & SUMAS WAY,2020,1.0,1.0,2.0,ABBOTSFORD
3,4TH AVE & SUMAS WAY,2021,0.0,4.0,4.0,ABBOTSFORD
4,4TH AVE & SUMAS WAY,2022,3.0,5.0,8.0,ABBOTSFORD
...,...,...,...,...,...,...
560,TRANS-CANADA HWY & WHATCOM RD & WHATCOM RD OFF...,2018,34.0,45.0,79.0,ABBOTSFORD
561,TRANS-CANADA HWY & WHATCOM RD & WHATCOM RD OFF...,2019,58.0,60.0,118.0,ABBOTSFORD
562,TRANS-CANADA HWY & WHATCOM RD & WHATCOM RD OFF...,2020,19.0,36.0,55.0,ABBOTSFORD
563,TRANS-CANADA HWY & WHATCOM RD & WHATCOM RD OFF...,2021,6.0,9.0,15.0,ABBOTSFORD


In [303]:
#Repeat and concat all remaining data frames for the other municipalities

#Create list of remaining municipalities to include
list = ['Burnaby', 'Coquitlam', 'Delta', 'Langley','MapleRidge','NorthVancouver','PittMeadows','PortCoquitlam','Richmond','Surrey','Vancouver']
#Create list of labels to apply to columns
labels = ['BURNABY', 'COQUITLAM', 'DELTA', 'LANGLEY','MAPLE RIDGE','NORTH VANCOUVER','PITT MEADOWS','PORT COQUITLAM','RICHMOND','SURREY','VANCOUVER']

for i in range(11):
    #Create string for data frame creation and read in
    string = "data/"+list[i]+".csv"
    df=pd.read_csv(string)

    #Remove any entries not for intersections, and drop columns with latitude and longitude data
    mask = df['Location'].str.contains('&')
    df = df[mask]
    df.drop(['Latitude', 'Longitude'], axis=1, inplace=True)

    #Include only crashes that are classified as "Property Damage Only"
    mask = df['Crash Type'].str.contains('PDO')
    df_pdo = df[mask]

    #Filter again to only contain those with at least 40 crashes across the entire sample

    #Find all intersection names
    unique_names = df_pdo['Location'].unique()
    output = [None]*len(unique_names)
    for j in range(len(unique_names)):
        output[j] = df_pdo.loc[df['Location'] == unique_names[j], 'Crash Count'].sum()

    #temporary data frame that stores names to be filtered
    temp = pd.DataFrame({'Unique Names': unique_names, 'Total Crashes': output})
    temp['Condition'] = temp['Total Crashes'] >= condition

    #filtering rows containing these names from the df_pdo
    names = temp.loc[~temp['Condition'], 'Unique Names']
    df_pdo = df_pdo[~df_pdo['Location'].isin(names)]

    #Filter to only contain crash type "Casualty" and only include intersections that were included under the PDO criteria
    mask = df['Crash Type'].str.contains('Casualty')
    df_casualty = df[mask]
    df_casualty = df_casualty[df_casualty['Location'].isin(df_pdo['Location'])]


    #Find missing rows for pdo
    all_combinations_pdo = pd.DataFrame([(location, year) for location in df_pdo['Location'].unique() for year in df_pdo['Year'].unique()], columns=['Location', 'Year'])

    # Merge the new DataFrame with the original DataFrame to identify missing combinations
    merged_df_pdo = pd.merge(all_combinations_pdo, df_pdo, on=['Location', 'Year'], how='left', indicator=True)

    # Filter rows where the original DataFrame had missing data (indicator column is 'left_only')
    missing_data_pdo = merged_df_pdo[merged_df_pdo['_merge'] == 'left_only']

    # Drop the indicator column
    missing_data_pdo = missing_data_pdo.drop(columns=['_merge'])


    #Repeat above for casualty
    all_combinations_casualty = pd.DataFrame([(location, year) for location in df_casualty['Location'].unique() for year in df_casualty['Year'].unique()], columns=['Location', 'Year'])

    # Merge the new DataFrame with the original DataFrame to identify missing combinations
    merged_df_casualty = pd.merge(all_combinations_casualty, df_casualty, on=['Location', 'Year'], how='left', indicator=True)

    # Filter rows where the original DataFrame had missing data (indicator column is 'left_only')
    missing_data_casualty = merged_df_casualty[merged_df_casualty['_merge'] == 'left_only']

    # Drop the indicator column
    missing_data_casualty = missing_data_casualty.drop(columns=['_merge'])


    #Imputing new rows with zero crashes in the relevant 
    missing_data_pdo['Crash Type'].fillna('PDO', inplace=True)
    missing_data_pdo['Municipality'].fillna(labels[i], inplace=True)
    missing_data_pdo['Crash Count'].fillna(0, inplace=True)

    missing_data_casualty['Crash Type'].fillna('Casualty', inplace=True)
    missing_data_casualty['Municipality'].fillna(labels[i], inplace=True)
    missing_data_casualty['Crash Count'].fillna(0, inplace=True)

    #Concatonate imputed rows into data frame
    df = pd.concat([df_pdo,df_casualty,missing_data_pdo,missing_data_casualty],axis=0).sort_values(by='Location')

    #Pivot df to sort crash counts
    pivot_df = df.pivot(index=['Location', 'Year'], columns='Crash Type', values='Crash Count').reset_index()
    pivot_df.columns = ['Location', 'Year', 'Casualty', 'PDO']

    #Add columns back for municipality, and total number of crashes
    pivot_df['Total']=pivot_df['PDO']+pivot_df['Casualty']
    pivot_df['Municipality'] = labels[i]

    #Add the newly constructed df to the main df
    df_main = pd.concat([df_main,pivot_df])

df_main

,Location,Year,Casualty,PDO,Total,Municipality
0,4TH AVE & SUMAS WAY,2018,1.0,2.0,3.0,ABBOTSFORD
1,4TH AVE & SUMAS WAY,2019,2.0,8.0,10.0,ABBOTSFORD
2,4TH AVE & SUMAS WAY,2020,1.0,1.0,2.0,ABBOTSFORD
3,4TH AVE & SUMAS WAY,2021,0.0,4.0,4.0,ABBOTSFORD
4,4TH AVE & SUMAS WAY,2022,3.0,5.0,8.0,ABBOTSFORD
...,...,...,...,...,...,...
3905,W BROADWAY & YUKON ST,2018,12.0,14.0,26.0,VANCOUVER
3906,W BROADWAY & YUKON ST,2019,8.0,17.0,25.0,VANCOUVER
3907,W BROADWAY & YUKON ST,2020,7.0,7.0,14.0,VANCOUVER
3908,W BROADWAY & YUKON ST,2021,4.0,10.0,14.0,VANCOUVER


In [304]:
#Create dummies for each year and for each municipality
years = pd.get_dummies(df_main['Year'])*1

#add dummy columns for years and delete year column from data frame
df_main = pd.concat([df_main,years],axis=1).drop(['Year'],axis=1)

df_main

,Location,Casualty,PDO,Total,Municipality,2018,2019,2020,2021,2022
0,4TH AVE & SUMAS WAY,1.0,2.0,3.0,ABBOTSFORD,1,0,0,0,0
1,4TH AVE & SUMAS WAY,2.0,8.0,10.0,ABBOTSFORD,0,1,0,0,0
2,4TH AVE & SUMAS WAY,1.0,1.0,2.0,ABBOTSFORD,0,0,1,0,0
3,4TH AVE & SUMAS WAY,0.0,4.0,4.0,ABBOTSFORD,0,0,0,1,0
4,4TH AVE & SUMAS WAY,3.0,5.0,8.0,ABBOTSFORD,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
3905,W BROADWAY & YUKON ST,12.0,14.0,26.0,VANCOUVER,1,0,0,0,0
3906,W BROADWAY & YUKON ST,8.0,17.0,25.0,VANCOUVER,0,1,0,0,0
3907,W BROADWAY & YUKON ST,7.0,7.0,14.0,VANCOUVER,0,0,1,0,0
3908,W BROADWAY & YUKON ST,4.0,10.0,14.0,VANCOUVER,0,0,0,1,0


In [305]:
#Create dummies for each year
df_2018 = df_main[df_main[2018] == 1].groupby('Location').first().reset_index()
df_2019 = df_main[df_main[2019] == 1].groupby('Location').first().reset_index()
df_2020 = df_main[df_main[2020] == 1].groupby('Location').first().reset_index()
df_2021 = df_main[df_main[2021] == 1].groupby('Location').first().reset_index()
df_2022 = df_main[df_main[2022] == 1].groupby('Location').first().reset_index()

df_main = pd.concat([df_2018,df_2019,df_2020,df_2021,df_2022]).sort_values(by='Location')

df_main.reset_index(inplace=True)

df_main.drop(['index'], axis=1, inplace=True)

df_main



,Location,Casualty,PDO,Total,Municipality,2018,2019,2020,2021,2022
0,1 AVE & 176 ST,1.0,4.0,5.0,SURREY,1,0,0,0,0
1,1 AVE & 176 ST,0.0,0.0,0.0,SURREY,0,0,0,1,0
2,1 AVE & 176 ST,1.0,1.0,2.0,SURREY,0,0,1,0,0
3,1 AVE & 176 ST,3.0,12.0,15.0,SURREY,0,1,0,0,0
4,1 AVE & 176 ST,1.0,3.0,4.0,SURREY,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
10350,W BROADWAY & YUKON ST,4.0,10.0,14.0,VANCOUVER,0,0,0,1,0
10351,W BROADWAY & YUKON ST,12.0,14.0,26.0,VANCOUVER,1,0,0,0,0
10352,W BROADWAY & YUKON ST,7.0,7.0,14.0,VANCOUVER,0,0,1,0,0
10353,W BROADWAY & YUKON ST,8.0,17.0,25.0,VANCOUVER,0,1,0,0,0


In [306]:
#Creating data frame for total accidents to use for the computation of relative fraction of accidents
data = {'2018': [206967,48042],
        '2019': [196496,45947],
        '2020': [139718,30716],
        '2021': [166777,36116],
        '2022': [184894,35195]}

df_totals = pd.DataFrame(data)

PDO = abs(df_totals.diff().iloc[-1])

df_totals = df_totals.append(PDO, ignore_index=True)
df_totals.rename(index={0: 'Total Crashes', 1: 'Casualty Crashes', 2: 'PDO Crashes'}, inplace=True)

df_totals



C:\Users\seamu\AppData\Local\Temp\ipykernel_3924\363645931.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_totals = df_totals.append(PDO, ignore_index=True)


,2018,2019,2020,2021,2022
Total Crashes,206967.0,196496.0,139718.0,166777.0,184894.0
Casualty Crashes,48042.0,45947.0,30716.0,36116.0,35195.0
PDO Crashes,158925.0,150549.0,109002.0,130661.0,149699.0


In [307]:
#Adding new row that is the ratio of total crashes by intersection
for year in range (2018,2023):
    condition = df_main[year] == 1
    # df_main['PDOFrac'] = df_main['PDO Crash Count'].apply(lambda x: x/df_totals.at['PDO Crashes',year] if condition else x)
    df_main.loc[condition, 'PDOFrac'] = df_main.loc[condition, 'PDO'] / df_totals.at['PDO Crashes',str(year)]  # Division only for rows where the condition is True
    df_main.loc[condition, 'CasualtyFrac'] = df_main.loc[condition, 'Casualty'] / df_totals.at['Casualty Crashes',str(year)]  # Division only for rows where the condition is True
    df_main.loc[condition, 'TotalFrac'] = df_main.loc[condition, 'Total'] / df_totals.at['Total Crashes',str(year)]  # Division only for rows where the condition is True

df_main

,Location,Casualty,PDO,Total,Municipality,2018,2019,2020,2021,2022,PDOFrac,CasualtyFrac,TotalFrac
0,1 AVE & 176 ST,1.0,4.0,5.0,SURREY,1,0,0,0,0,0.000025,0.000021,0.000024
1,1 AVE & 176 ST,0.0,0.0,0.0,SURREY,0,0,0,1,0,0.000000,0.000000,0.000000
2,1 AVE & 176 ST,1.0,1.0,2.0,SURREY,0,0,1,0,0,0.000009,0.000033,0.000014
3,1 AVE & 176 ST,3.0,12.0,15.0,SURREY,0,1,0,0,0,0.000080,0.000065,0.000076
4,1 AVE & 176 ST,1.0,3.0,4.0,SURREY,0,0,0,0,1,0.000020,0.000028,0.000022
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10350,W BROADWAY & YUKON ST,4.0,10.0,14.0,VANCOUVER,0,0,0,1,0,0.000077,0.000111,0.000084
10351,W BROADWAY & YUKON ST,12.0,14.0,26.0,VANCOUVER,1,0,0,0,0,0.000088,0.000250,0.000126
10352,W BROADWAY & YUKON ST,7.0,7.0,14.0,VANCOUVER,0,0,1,0,0,0.000064,0.000228,0.000100
10353,W BROADWAY & YUKON ST,8.0,17.0,25.0,VANCOUVER,0,1,0,0,0,0.000113,0.000174,0.000127


In [308]:
#Add additional column for treatment group
treatedgroup = ['LONZO RD & SUMAS WAY & SUMAS WAY OFFRAMP & SUMAS WAY ONRAMP & TRANS-CANADA HWY & TURNING LANE',
                'BOUNDARY RD & KINGSWAY & TURNING LANE',
                'KINGSWAY & ROYAL OAK AVE',
                'DEER LAKE PKY & MOSCROP ST & WILLINGDON AVE',
                'BARNET HWY & JOHNSON ST & MARINER WAY & TURNING LANE',
                '84 AVE & NORDEL WAY',
                '84 AVE & NORDEL WAY & TURNING LANE',
                '200 ST & 64 AVE',
                'FRASER HWY & LANGLEY BYPASS & TURNING LANE',
                '207 ST & LOUGHEED HWY',
                'CAPILANO RD & MARINE DR & MATHIAS RD & TURNING LANE',
                'KENNEDY RD & LOUGHEED HWY & OLD DEWDNEY TRUNK RD & TURNING LANE',
                'LOUGHEED HWY & SHAUGHNESSY ST & TURNING LANE',
                'CAMBIE RD & GARDEN CITY RD',
                '152 ST & KING GEORGE BLVD & TURNING LANE',
                '152 ST & 64 AVE',
                '80 AVE & KING GEORGE BLVD & TURNING LANE',
                '128 ST & 88 AVE',
                '132 ST & 96 AVE',
                '152 ST & 96 AVE',
                '104 AVE & KING GEORGE BLVD',
                'BOUNDARY RD & E 49TH AVE & IMPERIAL ST & TURNING LANE',
                'KERR ST & SE MARINE DR',
                'JOYCE ST & KINGSWAY',
                'GRANDVIEW HWY & RUPERT ST',
                'E HASTINGS ST & RENFREW ST',
                'KINGSWAY & VICTORIA DR',
                'E 33RD AVE & KNIGHT ST',
                'OAK ST & W 70TH AVE',
                'OAK ST & W 57TH AVE',
                'GRANVILLE ST & W KING EDWARD AVE',
                'E HASTINGS ST & MAIN ST',
                'CARDERO ST & W GEORGIA ST & W PENDER ST']

df_main['Treatment'] = 0  # Initialize 'treatment' to 0 for all rows
df_main['Treatment'][df_main['Location'].isin(treatedgroup)] = 1


C:\Users\seamu\AppData\Local\Temp\ipykernel_3924\2634977332.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_main['Treatment'][df_main['Location'].isin(treatedgroup)] = 1


In [309]:
#Add additional dummy variable for whether or not the intersection has a red light. This is the treatment group + all other standard redlight cameras
redlights = ['LONZO RD & SUMAS WAY & SUMAS WAY OFFRAMP & SUMAS WAY ONRAMP & TRANS-CANADA HWY & TURNING LANE',
                'BOUNDARY RD & KINGSWAY & TURNING LANE',
                'KINGSWAY & ROYAL OAK AVE',
                'DEER LAKE PKY & MOSCROP ST & WILLINGDON AVE',
                'BARNET HWY & JOHNSON ST & MARINER WAY & TURNING LANE',
                '84 AVE & NORDEL WAY',
                '84 AVE & NORDEL WAY & TURNING LANE',
                '200 ST & 64 AVE',
                'FRASER HWY & LANGLEY BYPASS & TURNING LANE',
                '207 ST & LOUGHEED HWY',
                'CAPILANO RD & MARINE DR & MATHIAS RD & TURNING LANE',
                'KENNEDY RD & LOUGHEED HWY & OLD DEWDNEY TRUNK RD & TURNING LANE',
                'LOUGHEED HWY & SHAUGHNESSY ST & TURNING LANE',
                'CAMBIE RD & GARDEN CITY RD',
                '152 ST & KING GEORGE BLVD & TURNING LANE',
                '152 ST & 64 AVE',
                '80 AVE & KING GEORGE BLVD & TURNING LANE',
                '128 ST & 88 AVE',
                '132 ST & 96 AVE',
                '152 ST & 96 AVE',
                '104 AVE & KING GEORGE BLVD',
                'BOUNDARY RD & E 49TH AVE & IMPERIAL ST & TURNING LANE',
                'KERR ST & SE MARINE DR',
                'JOYCE ST & KINGSWAY',
                'GRANDVIEW HWY & RUPERT ST',
                'E HASTINGS ST & RENFREW ST',
                'KINGSWAY & VICTORIA DR',
                'E 33RD AVE & KNIGHT ST',
                'OAK ST & W 70TH AVE',
                'OAK ST & W 57TH AVE',
                'GRANVILLE ST & W KING EDWARD AVE',
                'E HASTINGS ST & MAIN ST',
                'CARDERO ST & W GEORGIA ST & W PENDER ST'
                'MAIN ST & MOUNTAIN HWY',
                'DENMAN ST & W GEORGIA ST',
                'BURRARD ST & DAVIE ST',
                'GRANVILLE ST & MARPOLE AVE & W 16TH AVE',
                'MAIN ST & TERMINAL AVE',
                'E BROADWAY & MAIN ST',
                'CLARK DR & E HASTINGS ST',
                'CLARK DR & VENABLES ST',
                'E HASTINGS ST & NANAIMO ST',
                'DUNDAS ST & N NANAIMO ST & NANAIMO ST',
                'COMMERCIAL DR & E 1ST AVE',
                'COMMERCIAL DR & E BROADWAY',
                'E BROADWAY & NANAIMO ST',
                '1ST AVE & BOUNDARY RD & E 1ST AVE',
                'OAK ST & W 33RD AVE',
                'GRANVILLE ST & W 41ST AVE',
                'GRANVILLE ST & W 49TH AVE',
                'GRANVILLE ST & MILTON ST & SW MARINE DR',
                'OAK ST & W 33RD AVE',
                'OAK ST & W 41ST AVE',
                'OAK ST & W 49TH AVE',
                'CAMBIE ST & W 41ST AVE',
                'E 49TH AVE & MAIN ST',
                'E 49TH AVE & FRASER ST',
                'E 49TH AVE & KNIGHT ST',
                'MAIN ST & SE MARINE DR',
                'FRASER ST & SE MARINE DR',
                'ARGYLE ST & SE MARINE DR',
                'SE MARINE DR & VICTORIA DR',
                'HASTINGS ST & WILLINGDON AVE',
                'LOUGHEED HWY & WILLINGDON AVE',
                'KINGSWAY & WILLINGDON AVE & TURNING LANE',
                'GILLEY AVE & KINGSWAY',
                'BURRIS ST & CANADA WAY',
                'CANADA WAY & EDMONDS ST',
                '10TH AVE & CANADA WAY & EIGHTH ST & TENTH AVE',
                '10TH AVE & KINGSWAY & TENTH AVE & TWELFTH ST',
                'GAGLARDI WAY & LOUGHEED HWY & TURNING LANE',
                'BRUNETTE AVE & LOUGHEED HWY',
                'LOUGHEED HWY & PITT RIVER RD',
                'BARNET HWY & LOUGHEED HWY & PINETREE WAY & TURNING LANE'
                'GUILDFORD WAY & PINETREE WAY & TURNING LANE',
                'ARGUE ST & MARY HILL BYPASS & SHAUGHNESSY ST & TURNING LANE',
                'MARY HILL BYPASS & PITT RIVER RD & TURNING LANE',
                'HARRIS RD & LOUGHEED HWY & TURNING LANE',
                'DEWDNEY TRUNK RD & LOUGHEED HWY & MAPLE MEADOWS WAY & WEST ST & TURNING LANE',
                '203 ST & LOUGHEED HWY',
                'ALDERBRIDGE WAY & GARDEN CITY RD & TURNING LANE',
                'NO 3 RD & WESTMINSTER HWY',
                'GARDEN CITY RD & WESTMINSTER HWY',
                'BLUNDELL RD & GILBERT RD',
                'BLUNDELL RD & NO 3 RD',
                'ALDERBRIDGE WAY & HWY 91 & SHELL RD & TURNING LANE',
                'NO 5 RD & WESTMINSTER HWY & TURNING LANE',
                'CAMBIE RD & NO 5 RD',
                '112A AVE & 128 ST & BRIDGEVIEW DR & KING GEORGE BLVD & TURNING LANE',
                '108 AVE & KING GEORGE BLVD & TURNING LANE',
                '120 ST & 96 AVE',
                '120 ST & NORDEL WAY',
                '120 ST & 80 AVE',
                '120 ST & 72 AVE',
                '120 ST & 58 AVE & HWY 10 & TURNING LANE',
                '128 ST & 96 AVE',
                '128 ST & 72 AVE',
                '132 ST & 88 AVE',
                '96 AVE & KING GEORGE BLVD & TURNING LANE',
                '92 AVE & KING GEORGE BLVD & TURNING LANE',
                '88 AVE & KING GEORGE BLVD & TURNING LANE',
                '76 AVE & KING GEORGE BLVD & TURNING LANE',
                '72 AVE & KING GEORGE BLVD & TURNING LANE',
                '64 AVE & KING GEORGE BLVD & TURNING LANE',
                '140 ST & 88 AVE',
                '96 AVE & FRASER HWY & GREEN TIMBERS GREENWAY & TURNING LANE',
                '104 AVE & 152 ST',
                '100 AVE & 152 ST',
                '152 ST & FRASER HWY & TURNING LANE',
                '152 ST & 88 AVE',
                '152 ST & 56 AVE',
                '176 ST & 64 AVE',
                '208 ST & 88 AVE & TURNING LANE',
                'GLOVER RD & HWY 10',
                'GEORGE FERGUSON WAY & GLADWIN RD & TURNING LANE',
                'MARSHALL RD & MCCALLUM RD',
                'FRASER ST & KINGSWAY',
                'BURRARD ST & BURRARD ST BRIDGE & PACIFIC ST']

df_main['Redlight'] = 0  # Initialize 'Redlight' to 0 for all rows
df_main['Redlight'][df_main['Location'].isin(redlights)] = 1


C:\Users\seamu\AppData\Local\Temp\ipykernel_3924\3418384984.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_main['Redlight'][df_main['Location'].isin(redlights)] = 1


In [311]:
#Export data frame to csv for use in main notebook that will handle the empirical work
csv_file_path = 'data/dataset.csv'

df_main.to_csv(csv_file_path, index=False)
